In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPooling1D, LSTM, GRU, Bidirectional, Dense, Dropout,Flatten
from sklearn.preprocessing import MinMaxScaler
from keras.regularizers import l1
from keras.callbacks import EarlyStopping, TensorBoard, ReduceLROnPlateau, ModelCheckpoint
from keras.models import Model

2023-04-04 18:58:46.981259: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-04 18:58:47.255980: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-04 18:58:47.257730: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 18:58:48.687332: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import glob
all_f= glob.glob('../../'+'DataSets/'+'*/'+'*/'+'*_INDEX_*.csv')
li=[]
for file_name in all_f:
    df=pd.read_csv(file_name)
    li.append(df)
s_n=1
df=li[s_n]
df,all_f[s_n]
df=df.iloc[:,:]
df['DATE']=pd.to_datetime(df['DATE'])
df=df.rename(columns={'PM2.5':'PM'})
df['PM']=df['PM'].astype('float32')
df=df.reset_index()
df=df.drop(['index'], axis = 1) 
#df.drop(['level_0'], axis = 1, inplace = True) 

data=df.set_index('DATE')
data
data = data['PM'].values.reshape(-1,1)


In [3]:
# Scale data
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [4]:
# Split data into training and testing sets
train_size = int(len(data) * 0.8)
test_size = len(data) - train_size
train_data, test_data = data[0:train_size,:], data[train_size:len(data),:]

In [5]:
# Convert data into time series dataset
def create_dataset(dataset, look_back=24):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i+look_back), 0])
        Y.append(dataset[i+look_back, 0])
    return np.array(X), np.array(Y)


In [6]:
look_back = 24
train_X, train_Y = create_dataset(train_data, look_back)
test_X, test_Y = create_dataset(test_data, look_back)

In [7]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)
# Reshape input data for CNN
train_X = np.expand_dims(train_X, axis=2)
test_X = np.expand_dims(test_X, axis=2)

In [8]:
# Define CNN + BiLSTM + GRU model
input_layer = Input(shape=(look_back, 1))
input_layer

<KerasTensor: shape=(None, 24, 1) dtype=float32 (created by layer 'input_1')>

In [9]:
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same',bias_regularizer=l1(0.4))(input_layer)
max_pool1 = MaxPooling1D(pool_size=2)(conv1)
lstm1 = Bidirectional(LSTM(units=64, return_sequences=True,bias_regularizer=l1(0.4)))(max_pool1)
gru1 = GRU(units=32, activation='tanh',bias_regularizer=l1(0.4), return_sequences=True)(lstm1)
gru2 = GRU(units=32, activation='tanh')(gru1)
dropout = Dropout(0.2)(gru2)
flatten1 = Flatten()(dropout)
dense1 = Dense(units=1)(flatten1)
model = Model(inputs=input_layer, outputs=dense1)
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

2023-04-04 18:58:51.958908: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-04 18:58:51.961847: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-04 18:58:51.963247: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 24, 32)            128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 32)           0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 12, 128)          49664     
 l)                                                              
                                                                 
 gru (GRU)                   (None, 12, 32)            15552     
                                                                 
 gru_1 (GRU)                 (None, 32)                6336  

2023-04-04 18:58:52.783750: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-04 18:58:52.785983: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-04 18:58:52.787561: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [10]:
# Train model
model.fit(train_X, train_Y, epochs=200, batch_size=24,steps_per_epoch=32, verbose=1,callbacks=[es])

Epoch 1/200


2023-04-04 18:58:53.424846: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-04 18:58:53.428121: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-04 18:58:53.430517: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

32/32 [==============================] - 11s 18ms/step - loss: 50.4760
Epoch 2/200
32/32 [==============================] - 1s 21ms/step - loss: 48.8095
Epoch 3/200
32/32 [==============================] - 1s 17ms/step - loss: 47.1685
Epoch 4/200
32/32 [==============================] - 1s 18ms/step - loss: 45.5290
Epoch 5/200
32/32 [==============================] - 1s 18ms/step - loss: 43.8905
Epoch 6/200
32/32 [==============================] - 1s 18ms/step - loss: 42.2522
Epoch 7/200
32/32 [==============================] - 1s 18ms/step - loss: 40.6127
Epoch 8/200
32/32 [==============================] - 1s 18ms/step - loss: 38.9743
Epoch 9/200
32/32 [==============================] - 1s 18ms/step - loss: 37.3362
Epoch 10/200
32/32 [==============================] - 1s 18ms/step - loss: 35.6975
Epoch 11/200
32/32 [==============================] - 1s 18ms/step - loss: 34.0597
Epoch 12/200
32/32 [==============================] - 1s 18ms/step - loss: 32.4212
Epoch 13/200
32/32 [====

In [11]:
# Make predictions
train_predict = model.predict(train_X)
test_predict = model.predict(test_X)


2023-04-04 18:59:29.338611: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-04 18:59:29.341302: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-04 18:59:29.343418: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

384/384 [==============================] - 2s 6ms/step


In [ ]:
# Invert predictions to original scale
train_predict = scaler.inverse_transform(train_predict)
train_Y = scaler.inverse_transform([train_Y])
test_predict = scaler.inverse_transform(test_predict)
test_Y = scaler.inverse_transform([test_Y])

# Calculate RMSE
train_rmse = np.sqrt(np.mean((train_predict - train_Y)**2))
test_rmse = np.sqrt(np.mean((test_predict - test_Y)**2))

print('Train RMSE:', train_rmse)
print('Test RMSE:', test_rmse)